In [ ]:
#第1步：加载google硬盘
from google.colab import drive
drive.mount('/content/drive')

In [1]:
#第2步：导入YouTube库
pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.7 MB/s eta 0:00:00


In [ ]:
#第3步（选一）：读取csv文件
import yt_dlp
import pandas as pd
from datetime import date
import os

# 定义函数：获取指定 YouTube 频道下的视频链接和信息
def get_video_links(channel_url):
    # yt-dlp 的配置选项
    ydl_opts = {
        'quiet': True,  # 静默模式，不输出详细信息
        'extract_flat': True,  # 仅提取信息，不下载视频
    }

    try:
        # 使用 yt-dlp 获取频道信息
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(channel_url, download=False)
            entries = info_dict.get('entries', [])
            # 从获取到的信息中提取视频标题和 URL
            video_info = [(entry.get('title', 'Untitled'), entry.get('url')) for entry in entries if entry.get('url')]
            # 获取上传者名称
            uploader = info_dict.get('uploader', 'UnknownUploader')
            return video_info, uploader
    except Exception as e:
        # 捕获异常并打印错误信息
        print(f"获取失败: {channel_url}\n错误: {e}")
        return [], "ErrorUploader"

# 定义函数：创建 .dpl 播放列表文件
def create_dpl_file(video_info, uploader, output_dir):
    # 获取今天的日期
    today_date = date.today().strftime("%Y-%m-%d")
    # 清理上传者名称，移除可能导致文件路径问题的字符
    safe_uploader = uploader.replace('/', '_').replace('\\', '_').replace(':', '_')
    # 构建文件名
    file_name = f"{safe_uploader}-{today_date}.dpl"
    # 构建完整的文件路径
    file_path = os.path.join(output_dir, file_name)

    # 以写入模式打开 .dpl 文件
    with open(file_path, 'w', encoding='utf-8') as f:
        # 写入播放列表文件头
        f.write("DAUMPLAYLIST\n")

        # 遍历视频信息，写入每个视频的条目
        for i, (title, url) in enumerate(video_info, start=1):
            f.write(f"{i}*file*{url}\n")  # 写入视频 URL
            f.write(f"{i}*title*{title}\n") # 写入视频标题

            # 提取视频ID（UID）用于生成缩略图链接
            uid = ""
            if "watch?v=" in url:
                uid = url.split("watch?v=")[-1].split("&")[0]
            elif "youtu.be/" in url:
                uid = url.split("youtu.be/")[-1].split("?")[0]

            # 写入缩略图链接，如果无法提取 UID 则写入 unknown
            if uid:
                f.write(f"{i}*thumbnail*https://i.ytimg.com/vi/{uid}/default.jpg\n")
            else:
                f.write(f"{i}*thumbnail*unknown\n")

    # 打印保存成功的消息
    print(f"✅ 已保存: {file_path}")

# 主函数
def main():
    # 配置CSV文件路径
    csv_path = r"/content/drive/MyDrive/Takeout/订阅内容.csv"
    # 获取今天的日期，用于创建输出文件夹名称
    today_date = date.today().strftime("%Y-%m-%d")
    # 构建输出文件夹路径 (Google Drive 路径)
    output_dir = f"/content/drive/MyDrive/Youtube_ subscription_to_dpl/YouTube订阅/{today_date}"

    # 创建输出文件夹（如果不存在）
    os.makedirs(output_dir, exist_ok=True)

    # 检查CSV文件是否存在
    if not os.path.exists(csv_path):
        print(f"文件不存在: {csv_path}")
        return

    try:
        # 使用 pandas 读取CSV文件
        # header=None: CSV没有列头
        # skiprows=1: 跳过第一行
        # usecols=[1]: 只读取第二列 (索引为1)
        # dtype=str: 确保读取的数据是字符串类型
        df = pd.read_csv(csv_path, header=None, skiprows=1, usecols=[1], dtype=str)
        # 将读取到的第二列数据（URL）转换为列表，并移除空值
        urls = df[1].dropna().tolist()
    except Exception as e:
        # 捕获CSV读取错误并打印信息
        print(f"CSV读取失败: {e}")
        return

    # 遍历每个频道链接
    for url in urls:
        # 确保 URL 格式正确，指向频道的视频页面
        full_url = url.rstrip('/') + '/videos'
        print(f"\n📥 正在处理: {full_url}")
        # 调用函数获取视频信息
        video_info, uploader = get_video_links(full_url)

        # 如果获取到视频信息，则创建 .dpl 文件
        if video_info:
            create_dpl_file(video_info, uploader, output_dir)
        else:
            # 如果没有获取到视频信息，打印提示
            print("⚠️ 无视频或处理失败。")

# 当脚本作为主程序运行时，执行 main 函数
if __name__ == "__main__":
    main()

In [2]:
#第3步（选一）：读取压缩包里面的csv文件
import yt_dlp
import pandas as pd
from datetime import date
import os
import zipfile

def get_video_links(channel_url):
    """
    使用 yt-dlp 获取指定 YouTube 频道下的所有视频链接和信息。
    """
    ydl_opts = {
        'quiet': True, # 静默模式，不输出详细信息
        'extract_flat': True,  # 仅提取信息，不下载视频
        'force_generic_extractor': True, # 强制使用通用提取器
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # 提取频道信息
            info_dict = ydl.extract_info(channel_url, download=False)
            entries = info_dict.get('entries', [])
            # 过滤掉None的entry并提取视频标题和URL
            video_info = [(entry.get('title', 'Untitled'), entry.get('url')) for entry in entries if entry and entry.get('url')]
            # 获取上传者名称
            uploader = info_dict.get('uploader', 'UnknownUploader')
            return video_info, uploader
    except Exception as e:
        # 捕获并打印获取视频列表时的错误
        print(f"获取视频列表失败: {channel_url}\n错误: {e}")
        return [], "ErrorUploader"

def create_dpl_file(video_info, uploader, output_dir):
    """
    为指定频道的视频列表创建一个 .dpl 播放列表文件。
    """
    # 获取今天的日期
    today_date = date.today().strftime("%Y-%m-%d")
    # 清理上传者名称，使其成为安全的文件名
    safe_uploader = "".join([c for c in uploader if c.isalnum() or c in (' ', '-')]).rstrip()
    # 构建文件名
    file_name = f"{safe_uploader}-{today_date}.dpl"
    # 构建完整的文件路径
    file_path = os.path.join(output_dir, file_name)

    # 以写入模式打开 .dpl 文件
    with open(file_path, 'w', encoding='utf-8') as f:
        # 写入播放列表文件头
        f.write("DAUMPLAYLIST\n")

        # 遍历视频信息并写入每个视频的条目
        for i, (title, url) in enumerate(video_info, start=1):
            f.write(f"{i}*file*{url}\n") # 写入视频 URL
            f.write(f"{i}*title*{title}\n") # 写入视频标题

            # 提取视频ID（UID）用于生成缩略图链接
            uid = ""
            if "watch?v=" in url:
                uid = url.split("watch?v=")[-1].split("&")[0]
            # 支持 youtu.be 格式的 URL
            elif "youtu.be/" in url:
                 uid = url.split("youtu.be/")[-1].split("?")[0]


            # 写入缩略图链接，如果无法提取 UID 则写入 unknown
            if uid:
                f.write(f"{i}*thumbnail*https://i.ytimg.com/vi/{uid}/default.jpg\n")
            else:
                f.write(f"{i}*thumbnail*unknown\n")

    print(f"✅ 播放列表已保存: {file_path}")

def main():
    # 1. 配置压缩包路径和要查找的CSV文件名
    zip_path = r"/content/drive/MyDrive/Takeout/takeout-20250713T185350Z-1-001.zip"
    csv_filename_in_zip = "订阅内容.csv"

    # 获取今天的日期，用于创建输出文件夹名称
    today_date = date.today().strftime("%Y-%m-%d")
    # 构建输出文件夹路径 (Google Drive 路径)
    output_dir = f"/content/drive/MyDrive/Youtube_ subscription_to_dpl/YouTube订阅/{today_date}"

    # 创建输出文件夹（如果不存在）
    os.makedirs(output_dir, exist_ok=True)

    urls = []
    # 2. 从Zip文件中读取CSV
    try:
        # 打开Zip文件
        with zipfile.ZipFile(zip_path, 'r') as z:
            # 在zip文件中查找CSV文件
            csv_path_in_zip = None
            for file_info in z.infolist():
                # 从完整路径中获取文件名并比较
                if os.path.basename(file_info.filename) == csv_filename_in_zip:
                    csv_path_in_zip = file_info.filename
                    break

            # 如果找到CSV文件
            if csv_path_in_zip:
                print(f"在压缩包中找到文件: {csv_path_in_zip}")
                # 直接从zip文件中打开并读取CSV
                with z.open(csv_path_in_zip) as csv_file:
                    # 使用 pandas 读取CSV，跳过第一行，只读取第二列
                    df = pd.read_csv(csv_file, header=None, skiprows=1, usecols=[1], dtype=str)
                    # 将读取到的第二列数据（URL）转换为列表，并移除空值
                    urls = df[1].dropna().tolist()
            else:
                # 如果未找到CSV文件，打印错误信息并返回
                print(f"错误: 在压缩包 '{zip_path}' 中未找到 '{csv_filename_in_zip}'。")
                return

    except FileNotFoundError:
        # 捕获并打印压缩包文件不存在的错误
        print(f"错误: 压缩包文件不存在: {zip_path}")
        return
    except Exception as e:
        # 捕获并打印读取压缩包或CSV文件时的错误
        print(f"读取压缩包或CSV文件时出错: {e}")
        return

    # 如果未能从CSV加载任何URL，打印提示并返回
    if not urls:
        print("未能从CSV文件中加载任何URL。")
        return

    # 遍历每个频道链接
    for url in urls:
        # 跳过空白URL
        if not url.strip():
            continue

        # 确保URL指向视频标签页，以获取所有视频
        full_url = url.rstrip('/') + '/videos'
        print(f"\n📥 正在处理频道: {full_url}")
        # 调用函数获取视频信息
        video_info, uploader = get_video_links(full_url)

        # 如果获取到视频信息，则创建 .dpl 文件
        if video_info:
            create_dpl_file(video_info, uploader, output_dir)
        else:
            # 如果未能获取视频列表，打印提示
            print(f"⚠️ 未能获取频道 '{uploader}' 的视频列表，或该频道无视频。")

# 当脚本作为主程序运行时，执行 main 函数
if __name__ == "__main__":
    main()

在压缩包中找到文件: Takeout/YouTube 和 YouTube Music/订阅内容/订阅内容.csv

📥 正在处理频道: http://www.youtube.com/channel/UC-q5vNvj81OmWK0pzaLB-kg/videos
✅ 播放列表已保存: /content/drive/MyDrive/YouTube订阅/2025-07-13/Sprouht-2025-07-13.dpl

📥 正在处理频道: http://www.youtube.com/channel/UC-qsP49Ai2GymJgyKX38l1w/videos
✅ 播放列表已保存: /content/drive/MyDrive/YouTube订阅/2025-07-13/Robert Greene-2025-07-13.dpl

📥 正在处理频道: http://www.youtube.com/channel/UC12ZjUpS9JB9O5oh3ul8jgQ/videos
✅ 播放列表已保存: /content/drive/MyDrive/YouTube订阅/2025-07-13/平凡以上 - 好用軟體推薦 WordPress-2025-07-13.dpl

📥 正在处理频道: http://www.youtube.com/channel/UC7JkWjbQOFicrYEm1tMZm9Q/videos
✅ 播放列表已保存: /content/drive/MyDrive/YouTube订阅/2025-07-13/苔科技MiniMossTech-2025-07-13.dpl

📥 正在处理频道: http://www.youtube.com/channel/UCC9X5bLhdMeLvPxlMb-GShw/videos
✅ 播放列表已保存: /content/drive/MyDrive/YouTube订阅/2025-07-13/大问题Dialectic-2025-07-13.dpl

📥 正在处理频道: http://www.youtube.com/channel/UCDHiLKI3IDMeNqbcKfAu6rQ/videos
✅ 播放列表已保存: /content/drive/MyDrive/YouTube订阅/2025-07-13/賴宇凡Sara-2025-07-13.